# 1. Sprint 11 - Selección de Región Petrolera en OilyGiant

En este proyecto se busca determinar cuál de tres regiones disponibles ofrece las mejores condiciones para abrir **200 nuevos pozos de petróleo**.

El análisis se basará en la construcción de un modelo de **regresión lineal** que permita predecir el volumen de reservas de cada pozo, seleccionar los más rentables y calcular la ganancia esperada.

Además, se aplicará la técnica de **bootstrapping** para evaluar el beneficio promedio, el intervalo de confianza y el riesgo de pérdidas, con el fin de tomar una decisión fundamentada y alineada con los objetivos de negocio.

## 1.1 Objetivo del Proyecto y Contexto

La compañía **OilyGiant** necesita expandir su producción y busca identificar la región más prometedora para la perforación de nuevos pozos.

El desafío consiste en seleccionar la región que ofrezca el **mayor beneficio esperado** al perforar 200 pozos, garantizando que el **riesgo de pérdidas no supere el 2.5%**.

Para resolver este problema se dispone de datos de exploración geológica de tres regiones, con parámetros técnicos de cada pozo y su volumen estimado de reservas.

Estos datos serán la base para construir y validar modelos predictivos, calcular beneficios potenciales y evaluar riesgos, con el objetivo de recomendar una única región para la inversión.

## 1.2 Configuración Inicial

En esta subsección se importan las librerías necesarias para el análisis, modelado y visualización de datos.  
Además, se valida la versión de cada librería utilizada para asegurar la **reproducibilidad** del proyecto en cualquier entorno.  
También se establece un estilo uniforme para las gráficas que se generarán a lo largo del análisis.

In [2]:
"""
1.2 Configuración inicial
Propósito: Importar librerías necesarias y validar sus versiones.
"""

# 1) Manejo de datos
import pandas as pd
import numpy as np

# 2) Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# 3) Modelado y validación
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 4) Configuración de visualización
sns.set(style="whitegrid")  # estilo limpio con cuadrícula
plt.rcParams["figure.figsize"] = (10, 6)  # tamaño estándar de las figuras

# 5) Validación de versiones
print("✅ Librerías importadas y configuración inicial completada\n")
print(f"Pandas versión: {pd.__version__}")
print(f"NumPy versión: {np.__version__}")
print(f"Matplotlib versión: {plt.matplotlib.__version__}")
print(f"Seaborn versión: {sns.__version__}")

import sklearn
print(f"Scikit-Learn versión: {sklearn.__version__}")

✅ Librerías importadas y configuración inicial completada

Pandas versión: 2.3.2
NumPy versión: 2.2.5
Matplotlib versión: 3.10.5
Seaborn versión: 0.13.2
Scikit-Learn versión: 1.7.1


# 2. Carga y Exploración de Datasets

En esta sección se trabajará con los datos proporcionados para el proyecto.  
Se cargarán los tres archivos correspondientes a las regiones de exploración petrolera (`geo_data_0.csv`, `geo_data_1.csv`, `geo_data_2.csv`) y se realizará una revisión inicial de su estructura.  
El objetivo es garantizar que los datos estén completos, en el formato adecuado y listos para el proceso de modelado.  
Además, se validará la calidad de la información revisando dimensiones, tipos de datos, valores nulos y posibles duplicados, y se explorará de manera preliminar la distribución de la variable objetivo **`product`** (volumen de reservas en miles de barriles).

## 2.1 Cargar Datasets

En esta subsección se importan los tres archivos que contienen los datos de exploración geológica de las regiones evaluadas:  
- `geo_data_0.csv`  
- `geo_data_1.csv`  
- `geo_data_2.csv`  

Cada archivo incluye las características de los pozos (`f0`, `f1`, `f2`) y la variable objetivo `product`, que representa el volumen estimado de reservas de petróleo en miles de barriles.  
El propósito es verificar que los archivos se carguen correctamente y conocer la cantidad de registros disponibles en cada uno.


In [4]:
"""
2.1 Cargar datasets
Propósito: Importar los tres datasets de exploración y mostrar sus dimensiones iniciales.
"""

# Rutas de los datasets
DATA_PATH_0 = "../datasets/geo_data_0.csv"
DATA_PATH_1 = "../datasets/geo_data_1.csv"
DATA_PATH_2 = "../datasets/geo_data_2.csv"

# Cargar datasets en DataFrames
df_region_0 = pd.read_csv(DATA_PATH_0)
df_region_1 = pd.read_csv(DATA_PATH_1)
df_region_2 = pd.read_csv(DATA_PATH_2)

# Mostrar dimensiones de cada dataset
print("Dimensiones Región 0:", df_region_0.shape)
print("Dimensiones Región 1:", df_region_1.shape)
print("Dimensiones Región 2:", df_region_2.shape)

# Vista rápida de las primeras filas de cada dataset
print("\nPrimeras filas Región 0:")
display(df_region_0.head())

print("\nPrimeras filas Región 1:")
display(df_region_1.head())

print("\nPrimeras filas Región 2:")
display(df_region_2.head())


Dimensiones Región 0: (100000, 5)
Dimensiones Región 1: (100000, 5)
Dimensiones Región 2: (100000, 5)

Primeras filas Región 0:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



Primeras filas Región 1:


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305



Primeras filas Región 2:


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
